In [ ]:
from pathlib import Path
from azureml.core import Dataset, Datastore
from azureml.core import Workspace
import azureml.core
from azureml.core import Experiment
from azureml.widgets import RunDetails

# Connect to the Workspace

In [ ]:
# Load the workspace from the saved config file
ws = Workspace.from_config()
print(f'Ready to use Azure ML {azureml.core.VERSION} to work with {ws.name}')

# Upload and Register a Dataset

In [ ]:
blob_datastore = Datastore.get(ws, 'workspaceblobstore')

In [ ]:
blob_datastore.upload('/home/jeremy/Documents/data/quiddler', target_path='quiddler/', overwrite=True)

In [ ]:
file_dataset = Dataset.File.from_files(path=[(blob_datastore,'quiddler/')], validate=True)

In [ ]:
file_dataset.take(1).download()

In [ ]:
file_dataset.register(ws,'quiddler_dataset')

# Training

In [ ]:
ds = ws.datasets['quiddler_dataset']

In [ ]:
type(ds)

In [ ]:
experiment_name = 'quiddler-3'
exp = Experiment(workspace=ws, name=experiment_name)

In [ ]:
#compute_target = ws.compute_targets['jb-ml-gpu3']
compute_target = 'local'

In [ ]:
from azureml.core.script_run_config import ScriptRunConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

In [ ]:
myenv = Environment(name="quiddler-training-env")
myenv.docker.enabled = True
myenv.docker.base_image = "mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.2-cudnn8-ubuntu18.04"
conda_dep = CondaDependencies('training-env.yml')
myenv.python.conda_dependencies=conda_dep

In [ ]:
est = ScriptRunConfig(source_directory='source', 
                script='training.py',
                environment=myenv,
                arguments = ['--input_data', ds.as_named_input('quiddler_ds').as_download(),
                             '--epochs', 3, 
                             '--batch_size', 6,
                             '--image_size', 512],
                compute_target=compute_target)

In [ ]:
# Run the experiment
run = exp.submit(config=est)

# Show the run details while running
RunDetails(run).show()
run.wait_for_completion()

In [ ]:
run.get_metrics()

# Register the best model

In [ ]:
import pandas as pd

In [ ]:
runs = pd.DataFrame(data = [(r, r.id, r.get_metrics(name='clf_error').get('clf_error')) for r in exp.get_runs()],
                    columns = ['run', 'id','clf_error'])

In [ ]:
runs.dropna(inplace=True)

In [ ]:
runs.sort_values('clf_error')

In [ ]:
best_run = runs.run[0]

In [ ]:
best_run

In [ ]:
type(best_run)

In [ ]:
last_COCOMetric_mAP = best_run.get_metrics()['COCOMetric_mAP'][-1]

In [ ]:
from azureml.core import Model

# Register model
best_run.register_model(model_path='outputs/quiddler.pt', model_name='quiddler_model',
                        tags={'Training context':'IceVision on Compute Cluster'},
                        properties={'last_COCOMetric_mAP': last_COCOMetric_mAP})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')